In [11]:
import numpy as np
import argparse
import cv2
import os
from datetime import datetime
from PIL import Image
import pickle

In [2]:
def save_img(file_name, img, saveTo):
    if not os.path.exists(saveTo):
        os.makedirs(saveTo)
    img = img[:,:,::-1]
    img = Image.fromarray(img)

    try:
        img.save(saveTo + "/" + file_name + ".jpg", resolution=100.0)
    except Exception as ex:
        print("Error while saving file.")
        print(ex)
        return False
    else: return True

In [60]:

def capture_save_calibrate_images(camId = 0, name_prefix = 'Checkerboard_', path = f'{os.path.abspath(os.curdir)}\\checkerboards_cal'): 
    font                   = cv2.FONT_HERSHEY_DUPLEX
    position               = (15,50)
    fontScale              = 1.3
    fontColor              = (255,255,255)
    lineType               = 2

    cap = cv2.VideoCapture(camId)
    
    while True:
        ret, frame = cap.read()
        if ret is False: 
            return
        img = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)
        
        if corners is not None:
            for i in corners:
                x,y = i.ravel()
                cv2.circle(frame,(x,y),5,(0,0,255),-1)
                
        key =  cv2.waitKey(30) & 0xFF
        if key == ord('q'):
            print('quited')
            break
        elif key == ord('s'):
            cv2.putText(frame,'To save click S, to pass P', 
                position, 
                font, 
                fontScale,
                fontColor,
                lineType)
            cv2.imshow('frame', frame)
            doSave = cv2.waitKey(0)  & 0xFF
            if doSave == ord('s'):
                gene_name = f'{name_prefix}{datetime.now():%Y%m%d_%H%M%S}'
                res = save_img(gene_name, img, path) 
                if res is True: print(f"Saved: {os.path.abspath(os.curdir)}\\checkerboards_cal\\{gene_name}")
            else: continue
        cv2.imshow('frame', frame)
    cap.release
    cv2.destroyAllWindows()
        

In [61]:
capture_save_calibrate_images(1)

quited


In [5]:
def cal_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist, mtx, dist

In [12]:

def capture_save_undistort_img(camId = 0, name_prefix = 'work_space_photo', path = f'{os.path.abspath(os.curdir)}\work_space',
                                  calib_data_file = None): 
    font                   = cv2.FONT_HERSHEY_DUPLEX
    position               = (15,50)
    fontScale              = 1.3
    fontColor              = (255,255,255)
    lineType               = 2

    cap = cv2.VideoCapture(camId)
    
    do_undistort = False
    if calib_data_file is not None:
        try:
            data_file = open(calib_data_file, 'rb')
        except: 
                print('Calibration data Not loaded.')
                return
            
        calib_data = pickle.load(data_file)
        data_file.close()
        mtx = calib_data['mtx'][0]
        dist = calib_data['dist'][0]
        do_undistort = True
        print('Calibration data loaded.')
            
    while True:
        ret, frame = cap.read()
        if ret is False: 
            return
        if do_undistort is True:
            frame = cv2.undistort(frame,mtx,dist,None,mtx)
        img = frame.copy()
                
        key =  cv2.waitKey(30) & 0xFF
        if key == ord('q'):
            print('quited')
            break
        elif key == ord('s'):
            cv2.putText(frame,'To save click S, to pass P', 
                position, 
                font, 
                fontScale,
                fontColor,
                lineType)
            cv2.imshow('frame', frame)
            doSave = cv2.waitKey(0)  & 0xFF
            if doSave == ord('s'):
                gene_name = f'{name_prefix}{datetime.now():%Y%m%d_%H%M%S}'
                res = save_img(gene_name, img, path) 
                if res is True: print(f"Saved: {os.path.abspath(os.curdir)}\\checkerboards_cal\\{gene_name}")
            else: continue
        cv2.imshow('frame', frame)
    cap.release
    cv2.destroyAllWindows()
        

In [13]:
capture_save_undistort_img(1, calib_data_file='cam_calibration_data.p')

Calibration data loaded.
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\work_space_photo20181119_030253
quited


In [83]:
os.path.abspath(os.curdir)

'C:\\Users\\barte\\Documents\\Studia VII\\Image_processing\\Calibrate and sketch view'

In [98]:
data_file = open('cam_calibration_data.p', 'rb')
calib_data = pickle.load(data_file)
mtx = calib_data['mtx']
dist = calib_data['dist']
data_file.close()

In [100]:
np.array(mtx, dtype='uint8')

array([[[163,   0,  72],
        [  0, 166, 206],
        [  0,   0,   1]]], dtype=uint8)

In [101]:
mtx

(array([[2.21190685e+03, 0.00000000e+00, 3.28922080e+02],
        [0.00000000e+00, 2.21431413e+03, 2.06185061e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),)